![Snowflake](../img/snowflake-logo.png)
# Snowpark Quick Demo

This is a quick demo that aims to demonstrate how easy and simple is to get started with **Snowpark** and start processing data within the Snowflake Platform

In [1]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import udf, substr

## Inform your Snowflake connection data

In [2]:
snowflake_account = '' 
snowflake_user = 'MLINS'
snowflake_password = 'Mateus2014'
snowflake_user_role = 'SYSADMIN'
snowflake_warehouse = 'DEMO_WH'
snowflake_database = 'DEMO'
snowflake_schema = 'JSONDEMO'

In [3]:
connection_parameters = {
  "account": snowflake_account,
  "user": snowflake_user,
  "password": snowflake_password,
  "role": snowflake_user_role,
  "warehouse": snowflake_warehouse,
  "database": snowflake_database,
  "schema": snowflake_schema
}

## Create the Snowpark session

In [4]:
session = Session.builder.configs(connection_parameters).create()

## Checking the connection with the Snowflake environment

In [5]:
print(session.sql("select current_warehouse(), current_database(), current_schema()").collect())

[Row(CURRENT_WAREHOUSE()='DEMO_WH', CURRENT_DATABASE()='DEMO', CURRENT_SCHEMA()='JSONDEMO')]


## Visualising the JSON data that was mapped in an external stage 

In [6]:
session.read.table("JSON_RAW").show()

-------------------------------------------------
|"RAW_FILE"                                     |
-------------------------------------------------
|{                                              |
|  "body": [                                    |
|    {                                          |
|      "areaCode": "E40000003",                 |
|      "areaName": "London",                    |
|      "areaType": "nhsRegion",                 |
|      "date": "2022-07-20",                    |
|      "hospitalCases": 2303,                   |
|      "newAdmissions": null                    |
|    },                                         |
|    {                                          |
|      "areaCode": "E40000003",                 |
|      "areaName": "London",                    |
|      "areaType": "nhsRegion",                 |
|      "date": "2022-07-19",                    |
|      "hospitalCases": 2339,                   |
|      "newAdmissions": null                    |


## Querying and parsing the JSON to get all needed columns

In [7]:
df_nhs_covid = session.sql("SELECT value:areaCode::STRING as area_code," \
                                   "value:areaName::STRING as area_name," \
                                   "value:date::DATE as day_date," \
                                   "value:hospitalCases::INT as num_hospital_cases," \
                                   "value:newAdmissions::INT as num_admissions " \
                           "FROM DEMO.JSONDEMO.JSON_RAW, LATERAL FLATTEN( INPUT => raw_file:body )")

In [8]:
df_nhs_covid.show()

------------------------------------------------------------------------------------
|"AREA_CODE"  |"AREA_NAME"  |"DAY_DATE"  |"NUM_HOSPITAL_CASES"  |"NUM_ADMISSIONS"  |
------------------------------------------------------------------------------------
|E40000003    |London       |2022-07-20  |2303                  |NULL              |
|E40000003    |London       |2022-07-19  |2339                  |NULL              |
|E40000003    |London       |2022-07-18  |2330                  |246               |
|E40000003    |London       |2022-07-17  |2281                  |229               |
|E40000003    |London       |2022-07-16  |2282                  |179               |
|E40000003    |London       |2022-07-15  |2295                  |215               |
|E40000003    |London       |2022-07-14  |2257                  |278               |
|E40000003    |London       |2022-07-13  |2225                  |266               |
|E40000003    |London       |2022-07-12  |2237                  |

## Adding column

In [9]:
df_nhs_covid_month = df_nhs_covid.withColumn('month_year', substr(df_nhs_covid.DAY_DATE, 0, 7))

In [10]:
df_nhs_covid_month.show()

---------------------------------------------------------------------------------------------------
|"AREA_CODE"  |"AREA_NAME"  |"DAY_DATE"  |"NUM_HOSPITAL_CASES"  |"NUM_ADMISSIONS"  |"MONTH_YEAR"  |
---------------------------------------------------------------------------------------------------
|E40000003    |London       |2022-07-20  |2303                  |NULL              |2022-07       |
|E40000003    |London       |2022-07-19  |2339                  |NULL              |2022-07       |
|E40000003    |London       |2022-07-18  |2330                  |246               |2022-07       |
|E40000003    |London       |2022-07-17  |2281                  |229               |2022-07       |
|E40000003    |London       |2022-07-16  |2282                  |179               |2022-07       |
|E40000003    |London       |2022-07-15  |2295                  |215               |2022-07       |
|E40000003    |London       |2022-07-14  |2257                  |278               |2022-07       |


## Saving Dataframe as a new table within the Snowflake database

In [11]:
df_nhs_covid_month.write.mode('overwrite').save_as_table('NHS_COVID')

## Creating and registering an UDF function

In [12]:
@udf(name="calc_example", is_permanent=True, stage_location="@demo_stage", replace=True, session=session)
def calc_example(x: int, y: int) -> int:
    x = 1 if x is None else x
    y = 1 if y is None else y
    return round(x/y,2)


## Applying the created UDF to a new column

In [13]:
df_nhs_covid_calc = df_nhs_covid_month.withColumn('CALC', calc_example(df_nhs_covid_month.NUM_HOSPITAL_CASES, df_nhs_covid_month.NUM_ADMISSIONS))

In [14]:
df_nhs_covid_calc.show()

------------------------------------------------------------------------------------------------------------
|"AREA_CODE"  |"AREA_NAME"  |"DAY_DATE"  |"NUM_HOSPITAL_CASES"  |"NUM_ADMISSIONS"  |"MONTH_YEAR"  |"CALC"  |
------------------------------------------------------------------------------------------------------------
|E40000003    |London       |2022-07-20  |2303                  |NULL              |2022-07       |2303    |
|E40000003    |London       |2022-07-19  |2339                  |NULL              |2022-07       |2339    |
|E40000003    |London       |2022-07-18  |2330                  |246               |2022-07       |9       |
|E40000003    |London       |2022-07-17  |2281                  |229               |2022-07       |9       |
|E40000003    |London       |2022-07-16  |2282                  |179               |2022-07       |12      |
|E40000003    |London       |2022-07-15  |2295                  |215               |2022-07       |10      |
|E40000003    |Lond

## Applying aggreation

In [15]:
df_nhs_covid_month.group_by(df_nhs_covid_month.AREA_NAME).sum(df_nhs_covid_month.NUM_HOSPITAL_CASES).show()

--------------------------------------------------------
|"AREA_NAME"               |"SUM(NUM_HOSPITAL_CASES)"  |
--------------------------------------------------------
|London                    |1370918                    |
|Midlands                  |1364437                    |
|South East                |884465                     |
|South West                |463413                     |
|North West                |1141947                    |
|North East and Yorkshire  |1127033                    |
--------------------------------------------------------

